# 트랜스포머 인코더 시각화: 위치 인코딩

이 노트북은 `manim` 라이브러리를 사용하여 트랜스포머 모델 인코더의 위치 인코딩(Positional Encoding) 단계를 시각화합니다.

In [7]:
# 필요한 라이브러리 임포트
import numpy as np
from manim import *

# manim 환경 설정 (선택 사항, 필요에 따라 조정)
config.background_color = WHITE
config.frame_height = 6
config.frame_width = 10

In [9]:
%%manim -v WARNING -qh PositionalEncodingVisualization

class PositionalEncodingVisualization(Scene):
    def construct(self):
        # 씬 배경색 설정 (노트북 셀 배경과 유사하게)
        self.camera.background_color = WHITE

        # 제목
        title = Text("2. Positional Encoding", color=BLACK).to_edge(UP)
        self.play(Write(title))
        self.wait(0.5)

        # 임베딩 벡터 (이전 단계에서 생성되었다고 가정)
        embedding_dim = 4
        seq_len = 3
        vector_height = 1.0
        vector_width = 0.25

        embeddings = VGroup(*[
            VGroup(*[Rectangle(height=vector_height/embedding_dim, width=vector_width, fill_opacity=0.8, stroke_width=1, color=BLUE)
                     for _ in range(embedding_dim)]).arrange(DOWN, buff=0)
            for _ in range(seq_len)
        ]).arrange(RIGHT, buff=0.8).shift(UP*0.5)
        embedding_label = Text("Input Embeddings", color=BLACK, font_size=24).next_to(embeddings, UP, buff=0.3)

        self.play(FadeIn(embeddings), Write(embedding_label))
        self.wait(0.5)

        # 위치 인코딩 벡터 생성 (단순화된 시각화)
        # 실제로는 sin/cos 함수 사용
        positional_encodings = VGroup(*[
            VGroup(*[Rectangle(height=vector_height/embedding_dim, width=vector_width, fill_opacity=0.8, stroke_width=1, color=interpolate_color(GREEN, RED, i/seq_len))
                     for _ in range(embedding_dim)]).arrange(DOWN, buff=0)
            for i in range(seq_len)
        ]).arrange(RIGHT, buff=0.8).next_to(embeddings, DOWN, buff=1.5)
        pe_label = Text("Positional Encodings", color=BLACK, font_size=24).next_to(positional_encodings, UP, buff=0.3)

        # 위치 인코딩 벡터 등장 애니메이션
        pe_creation_anims = []
        for i, pe_vec in enumerate(positional_encodings):
            pos_text = Text(f"pos={i}", color=BLACK, font_size=18).next_to(pe_vec, DOWN, buff=0.2)
            pe_creation_anims.extend([FadeIn(pe_vec, shift=DOWN*0.2), Write(pos_text)])

        self.play(Write(pe_label))
        self.play(LaggedStart(*pe_creation_anims, lag_ratio=0.7))
        self.wait(0.5)

        # 덧셈 기호
        plus_signs = VGroup(*[MathTex("+", color=BLACK).scale(1.5).move_to(
            (embeddings[i].get_center() + positional_encodings[i].get_center()) / 2
        ) for i in range(seq_len)])

        self.play(LaggedStart(*[Write(p) for p in plus_signs], lag_ratio=0.5))
        self.wait(0.5)

        # 결과 벡터 (임베딩 + 위치 인코딩)
        result_embeddings = VGroup(*[
             VGroup(*[Rectangle(height=vector_height/embedding_dim, width=vector_width, fill_opacity=0.8, stroke_width=1, color=interpolate_color(BLUE, positional_encodings[i][0].get_color(), 0.5))
                     for _ in range(embedding_dim)]).arrange(DOWN, buff=0)
            for i in range(seq_len)
        ]).arrange(RIGHT, buff=0.8).next_to(positional_encodings, DOWN, buff=1.5)
        result_label = Text("Embeddings + Positional Encodings", color=BLACK, font_size=24).next_to(result_embeddings, UP, buff=0.3)

        # 결과 벡터 생성 애니메이션 (이동 및 결합)
        transform_anims = []
        for i in range(seq_len):
            # 각 벡터 그룹 복사
            emb_copy = embeddings[i].copy()
            pe_copy = positional_encodings[i].copy()
            # 결과 위치로 이동 및 결과 벡터로 변환
            transform_anims.append(Transform(emb_copy, result_embeddings[i]))
            transform_anims.append(Transform(pe_copy, result_embeddings[i])) # PE도 결과로 변환

        self.play(Write(result_label))
        # 원본 임베딩과 PE는 FadeOut, 복사본은 Transform
        self.play(
            LaggedStart(*transform_anims, lag_ratio=0.1),
            FadeOut(embeddings),
            FadeOut(positional_encodings),
            FadeOut(embedding_label),
            FadeOut(pe_label),
            FadeOut(VGroup(*[p.mobject for p in pe_creation_anims[1::2]])), # pos=i 텍스트 FadeOut (오류 수정)
            FadeOut(plus_signs)
        )
        # 결과 벡터 그룹을 중앙으로 이동
        self.play(result_embeddings.animate.move_to(ORIGIN), result_label.animate.next_to(result_embeddings, UP, buff=0.3))

        self.wait(2)

Manim Community v0.19.0

In [ ]:
%%manim -v WARNING -qh CircleAreaCalculation

class CircleAreaCalculation(Scene):
    def construct(self):
        # 원 생성
        circle = Circle(radius=2)
        circle.set_fill(BLUE, opacity=0.5)

        # 텍스트 생성
        radius_text = MathTex(r"r = 2")
        area_formula = MathTex(r"A = \pi r^2")
        area_result = MathTex(r"A = \pi \cdot 2^2 = 4\pi \approx 12.57")

        # 텍스트 배치
        radius_text.next_to(circle, DOWN)
        VGroup(area_formula, area_result).arrange(DOWN).next_to(circle, RIGHT)

        # 애니메이션 실행
        self.play(Create(circle))
        self.play(Write(radius_text))
        self.wait(1)
        self.play(Write(area_formula))
        self.wait(1)
        self.play(Write(area_result))
        self.wait(2)


Manim Community v0.19.0

In [14]:
%%manim -v WARNING -qh ExpWithBars

class ExpWithBars(Scene):
    def construct(self):
        # Title
        title = Text("Visualizing exp(score) in Attention", font_size=36, color=YELLOW)
        title.to_edge(UP, buff=0.5)
        self.play(Write(title))
        self.wait(0.5)

        # Narration 1
        narration1 = Paragraph(
            "We start with raw attention scores from the dot product of Query and Key.",
            font_size=24, width=10
        ).to_edge(DOWN, buff=0.5)
        self.play(Write(narration1))
        self.wait(1)

        # Raw scores
        raw_scores = [-1, 0, 1]
        bar_colors = [BLUE, ORANGE, GREEN]

        raw_bars = VGroup()
        raw_labels = VGroup()
        for i, score in enumerate(raw_scores):
            height = score + 2  # shift to make -1 visible (base = 1)
            rect = Rectangle(width=0.5, height=height, color=bar_colors[i], fill_opacity=0.7)
            rect.shift(DOWN * (1 - height / 2))  # adjust so all bars start at y= -1
            label = MathTex(str(score)).scale(0.8).next_to(rect, DOWN, buff=0.2)
            group = VGroup(rect, label)
            raw_bars.add(group)

        raw_bars.arrange(RIGHT, buff=1)
        raw_bars.shift(UP * 0.5)

        raw_label = Text("Raw Scores", font_size=28).next_to(raw_bars, UP, buff=0.3)
        self.play(Write(raw_label), Create(raw_bars))
        self.wait(1.5)

        # Narration 2
        self.play(FadeOut(narration1))
        narration2 = Paragraph(
            "Now we apply the exponential function to each score.",
            font_size=24, width=10
        ).to_edge(DOWN, buff=0.5)
        self.play(Write(narration2))
        self.wait(1.5)

        # exp(score) -> new heights
        exp_scores = [np.exp(s) for s in raw_scores]
        max_height = max(exp_scores)

        exp_bars = VGroup()
        exp_labels = VGroup()
        for i, val in enumerate(exp_scores):
            height = (val / max_height) * 3  # Normalize to max height = 3 for viewport
            rect = Rectangle(width=0.5, height=height, color=bar_colors[i], fill_opacity=0.7)
            rect.shift(DOWN * (1 - height / 2))  # anchor bottom at y = -1
            label = MathTex(r"\exp(" + str(raw_scores[i]) + r") = " + str(round(val, 2))).scale(0.8)
            label.next_to(rect, DOWN, buff=0.2)
            group = VGroup(rect, label)
            exp_bars.add(group)

        exp_bars.arrange(RIGHT, buff=1)
        exp_bars.shift(DOWN * 1.2)

        exp_label = Text("After exp(score)", font_size=28).next_to(exp_bars, UP, buff=0.3)

        # Animate transform from raw to exp
        self.play(Transform(raw_label, exp_label), Transform(raw_bars, exp_bars))
        self.wait(2)

        # Narration 3
        self.play(FadeOut(narration2))
        narration3 = Paragraph(
            "The exponential highlights large scores while shrinking smaller ones.",
            font_size=24, width=10
        ).to_edge(DOWN, buff=0.5)
        self.play(Write(narration3))
        self.wait(2)

        # Fade everything out
        self.play(
            FadeOut(title),
            FadeOut(exp_label),
            FadeOut(raw_bars),
            FadeOut(narration3)
        )
        self.wait()




Manim Community v0.19.0

In [ ]:
%%manim -v WARNING -qh RectangleColumns

class RectangleColumns(Scene):
    def construct(self):
        self.camera.background_color = WHITE # 배경 흰색으로 설정

        square_side = 0.8 # 정사각형 변 길이
        num_squares = 4
        column_buff = 0.2 # 열 내 사각형 간 간격
        inter_column_buff = 3 # 열 간 간격

        # 왼쪽 열 색상 (파란색 계열)
        left_colors = [BLUE_E, BLUE_D, BLUE_C, BLUE_B]
        # 오른쪽 열 색상 (초록색 계열)
        right_colors = [GREEN_E, GREEN_D, GREEN_C, GREEN_B]

        # 왼쪽 열 생성 (정사각형으로 변경)
        left_column = VGroup(*[
            Square(side_length=square_side, color=left_colors[i], fill_opacity=0.8, stroke_color=BLACK, stroke_width=2) # 테두리 추가
            for i in range(num_squares)
        ]).arrange(DOWN, buff=column_buff)

        # 오른쪽 열 생성 (정사각형으로 변경)
        right_column = VGroup(*[
            Square(side_length=square_side, color=right_colors[i], fill_opacity=0.8, stroke_color=BLACK, stroke_width=2) # 테두리 추가
            for i in range(num_squares)
        ]).arrange(DOWN, buff=column_buff)

        # 전체 그룹 생성 및 배치
        columns = VGroup(left_column, right_column).arrange(RIGHT, buff=inter_column_buff)

        # 애니메이션
        self.play(Create(columns))
        self.wait(1) # 초기 대기 시간

        # --- 첫 번째 정사각형 분할 및 이동 ---
        source_square = left_column[0]
        source_color = source_square.get_color()
        small_square_side = square_side / 2

        # 작은 정사각형 4개 생성 (초기 위치는 원본 정사각형 내부)
        small_squares = VGroup()
        positions = [
            source_square.get_center() + UL * small_square_side / 2,
            source_square.get_center() + UR * small_square_side / 2,
            source_square.get_center() + DL * small_square_side / 2,
            source_square.get_center() + DR * small_square_side / 2,
        ]
        for pos in positions:
            small_sq = Square(side_length=small_square_side, color=source_color, fill_opacity=0.8, stroke_color=BLACK, stroke_width=1)
            small_sq.move_to(pos)
            small_squares.add(small_sq)

        # 원본 정사각형을 작은 정사각형 4개로 변환 (분할 효과)
        self.play(ReplacementTransform(source_square, small_squares), run_time=1)
        self.wait(0.5)

        # 작은 정사각형들을 오른쪽 열의 각 정사각형으로 이동
        move_anims = []
        for i in range(num_squares):
            move_anims.append(small_squares[i].animate.move_to(right_column[i].get_center()))

        self.play(LaggedStart(*move_anims, lag_ratio=0.2), run_time=1.5)
        self.wait(0.5)

        # 작은 정사각형들 사라짐
        self.play(FadeOut(small_squares))

        self.wait(2) # 최종 대기 시간

Manim Community v0.19.0

In [ ]:
# 이전 셀에서 생성된 비디오 또는 마지막 프레임 표시
# %%manim 매직 명령은 기본적으로 비디오 파일을 생성하고 표시합니다.
# 만약 마지막 프레임 이미지만 필요하다면 %%manim 명령에 -s 플래그를 추가하고 아래 코드를 사용하세요.

# from IPython.display import Image, display
# import os
# image_path = os.path.join("media", "images", "trans_former", "PositionalEncodingVisualization_ManimCE_v0.18.1.png") # 파일명은 manim 버전에 따라 다를 수 있음
# if os.path.exists(image_path):
#     display(Image(filename=image_path))
# else:
#     print(f"이미지 파일을 찾을 수 없습니다: {image_path}")
#     print("이전 셀 실행 시 오류가 발생했거나 이미지 파일 경로가 다를 수 있습니다.")